In [4]:
import pandas as pd
import os
import numpy as np
import scipy.signal as sp
import matplotlib.pyplot as pl
import peakutils
from sklearn.decomposition import PCA
import datetime

In [6]:
%cd AL_120n_parallelised

/home/shruti/Documents/AL_120n_parallelised


In [1]:
%cd 2018-11-02_run1/

/home/shruti/Documents/AL_120n_parallelised/2018-11-02_run1


In [5]:
df = pd.read_csv('final_120n.csv')

In [6]:
df.axes

[RangeIndex(start=0, stop=268504, step=1),
 Index([u'time', u'n0v', u'n1v', u'n2v', u'n3v', u'n4v', u'n5v', u'n6v', u'n7v',
        u'n8v',
        ...
        u'n110v', u'n111v', u'n112v', u'n113v', u'n114v', u'n115v', u'n116v',
        u'n117v', u'n118v', u'n119v'],
       dtype='object', length=121)]

In [7]:
V = df.values[:,1:]
time = df.values[:,0]
time = time/1000
v_PN = V[:,:90]
v_LN = V[:,91:]

In [8]:
# which neurons are spiking? PN

num_spikingneurons = np.zeros(120)
for i in range(0,120):
    a,_ = sp.find_peaks(V[1000:,i], height = 35)
    if len(a):
        num_spikingneurons[i] = 1
print np.nonzero(num_spikingneurons)
print np.count_nonzero(num_spikingneurons)

spiking_neurons = np.array(np.nonzero(num_spikingneurons))
print spiking_neurons.shape

(array([23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
       41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57,
       58, 59, 60, 61, 62, 63, 64, 65, 66, 67]),)
44
(1, 44)


In [9]:
# which neurons are spiking? LN

num_spikingneurons_LN = np.zeros(120)
for i in range(90,120):
    a,_ = sp.find_peaks(V[1000:,i], height = -20)
    if len(a):
        num_spikingneurons_LN[i] = 1
print np.nonzero(num_spikingneurons_LN)
print np.count_nonzero(num_spikingneurons_LN)

spiking_neurons_LN = np.array(np.nonzero(num_spikingneurons_LN))
print spiking_neurons_LN.shape

(array([ 90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
       116, 117, 118, 119]),)
30
(1, 30)


In [10]:
#PN plot
%matplotlib
for i in range (0,5):
    pl.subplot(5,1,i+1)
    pl.plot(time,v_PN[:,spiking_neurons[0,i+9]])
    pl.title("Voltage PN - Number {}.png".format(spiking_neurons[0,i]))
    
pl.savefig("Figures/Voltage PN")

Using matplotlib backend: TkAgg


In [11]:
#LN plot

for i in range (0,5):
    pl.subplot(5,1,i+1)
    pl.plot(time,v_LN[:,spiking_neurons_LN[0,i+9]-90])
    pl.title("Voltage LN - Number {}.png".format(spiking_neurons_LN[0,i+9]))
    
pl.savefig("Figures/Voltage LN")

In [48]:
#plot PN voltage

%matplotlib
PN_no = 43
pl.plot(time,v_PN[:,PN_no])
pl.title("Voltage PN - Number {}.png".format(PN_no))
pl.savefig("Figures/Voltage PN - Number {}.png".format(PN_no))
#pl.plot(A,M[:,2])
#pl.plot(A,M[:,4])
#pl.plot(A,M[:,41])

Using matplotlib backend: TkAgg


In [1123]:
# plot LN voltage
pl.close()
LN_no = 12
pl.plot(time,v_LN[:,LN_no])
pl.title("Voltage LN - Number {}.png".format(LN_no))
pl.savefig("Figures/Voltage LN - Number {}.png".format(LN_no))

In [12]:
# plot LFP
pl.close()
C_avg = np.average(V[:,0:90], 1)
pl.plot(time,C_avg)
pl.grid(True)
pl.title('LFP')
manager = pl.get_current_fig_manager()
manager.resize(*manager.window.maxsize())

print len(C_avg)
pl.savefig('Figures/LFP.png')

268504


In [13]:
# preprocessing - demean and detrend 
C_avg = C_avg[50000:250000]
C_avg = C_avg - np.mean(C_avg)
C_avg = sp.detrend(C_avg)
len(C_avg)

200000

In [14]:
#Plot FFT (Power spectrum)
# pl.close()
pl.figure()
Fs = 100000.0
Ts = 1.0/Fs

n = len(C_avg)
k = np.arange(n)
T = n/Fs
frq = k/T
frq = frq[range(n/2)]
FFT_LFP = np.fft.fft(C_avg)/n
FFT_LFP = FFT_LFP[range(n/2)]
FFT_LFP = FFT_LFP**2
pl.plot(frq[3:200], abs(FFT_LFP[3:200])) #frq[0:x] frq goes upto x/time(in seconds)
pl.ylabel('FFT of LFP')
pl.xlabel('Frequency')
pl.title("FFT Spectrum of LFP")
pl.savefig('Figures/Frequency Spectrum of LFP.png')

In [1127]:
pl.close()

In [15]:
### raster Data 
rasterData_PN = np.zeros((300000,90))

for i in range(0,90):
    a,_ = sp.find_peaks(V[:,i], height = 35)
    rasterData_PN[a,i] = 1
    
print rasterData_PN.shape

rasterData_LN = np.zeros((300000,30))

for i in range(0,30):
    a,_ = sp.find_peaks(V[:,i+90], height = -20)
    rasterData_LN[a,i] = 1
    
print rasterData_LN.shape

(300000, 90)
(300000, 30)


In [ ]:
pl.close()
C_avg = np.average(V[:,spiking_neurons[0,:]], 1)
pl.plot(time,C_avg)
pl.grid(True)
pl.title('LFP')
manager = pl.get_current_fig_manager()
manager.resize(*manager.window.maxsize())
C_avg = C_avg - np.mean(C_avg)
C_avg = sp.detrend(C_avg)



In [873]:
pl.close()
Fs = 100000.0
Ts = 1.0/Fs

n = len(C_avg)
k = np.arange(n)
T = n/Fs
frq = k/T
frq = frq[range(n/2)]
FFT_LFP = np.fft.fft(C_avg)/n

FFT_LFP = FFT_LFP[range(n/2)]
pl.plot(frq[3:270], abs(FFT_LFP[3:270])) #frq[0:x] frq goes upto x/time(in seconds)
pl.ylabel('FFT of LFP')
pl.xlabel('Frequency')
pl.title("FFT Spectrum of LFP")

Text(0.5,1,'FFT Spectrum of LFP')

In [381]:
# plot - raster plot

#plot_neuron = [15, 18, 19, 21, 45]
#plot_neuron = spiking_neurons[0,:5]
#for p in range(0,len(plot_neuron)):
#    pl.subplot(len(plot_neuron),1,p+1)
#    pl.plot(np.arange(300000),rasterData[:,plot_neuron[p]])
#    pl.title("Neuron no - {}".format(plot_neuron[p]))

    
    
#manager = plt.get_current_fig_manager()
#manager.resize(*manager.window.maxsize())    
#pl.suptitle("Raster plots", fontsize=16)
#pl.savefig("Figures/Raster plots.png")


In [16]:
# Nice raster plot 
pl.spy(np.transpose(rasterData_PN), aspect='auto', markersize = 7)
pl.show()
manager = pl.get_current_fig_manager()
manager.resize(*manager.window.maxsize())

pl.title("Raster plot_PN")
pl.savefig("Figures/Raster plot_PN.png")

In [23]:
pl.spy(np.transpose(rasterData_LN), aspect='auto', markersize = 7)
pl.show()
manager = pl.get_current_fig_manager()
manager.resize(*manager.window.maxsize())

pl.title("Raster plot_LN")
pl.savefig("Figures/Raster plot_LN.png")

In [24]:
#PCA of the PN voltages

pca = PCA(n_components=10)
pc = pca.fit_transform(v_PN[:,spiking_neurons[0,:]])
pc.shape

(300000, 10)

In [817]:
pl.plot(pc[:,0], pc[:,1])

In [25]:
sum(pca.explained_variance_ratio_)

0.8364260549345861

In [ ]:
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y
